<a href="https://colab.research.google.com/github/Javed-ahmed-shanto/Deep-learning-for-developers/blob/main/ties4911_task05_shanto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Task 5-1**

In [ ]:
# # ALL-IN-ONE COMPARISON SCRIPT FOR GOOGLE COLAB
# # ----------------------------------------------
# # This single cell demonstrates how to:
# # 1) Install necessary libraries (Ultralytics YOLO, TorchVision)
# # 2) Download real images from the internet
# # 3) Load multiple object detection/segmentation models:
# #    - YOLOv8 (Ultralytics)
# #    - Faster R-CNN (TorchVision)
# #    - SSD (TorchVision)
# #    - RetinaNet (TorchVision)
# #    - Mask R-CNN (TorchVision)
# # 4) Run inference on ~5 real images
# # 5) Report performance (inference time) + a simple detection statistic
# # 6) Display bounding boxes from each model on one sample image
# #
# # NOTE:
# # - This script uses pretrained COCO weights for the TorchVision models.
# # - YOLOv8 model is also pretrained (default).
# # - We do not compute true accuracy metrics (mAP, IoU) because we do not have GT annotations.
# # - For more advanced usage, you would typically do:
# #   - Evaluate on a labeled dataset.
# #   - Possibly use GPU for faster inference.
# #
# # REFERENCES / IMPLEMENTATIONS:
# # - TorchVision Detection Models:
# #   https://pytorch.org/vision/stable/models/detection.html
# # - Ultralytics YOLO (v8+):
# #   https://github.com/ultralytics/ultralytics
# # - For additional models (U-Net, YOLACT, DETR, Transformers, etc.), see their respective repos:
# #   - U-Net: https://github.com/qubvel/segmentation_models.pytorch
# #   - YOLACT: https://github.com/dbolya/yolact
# #   - DETR: https://github.com/facebookresearch/detr

# ################################################################################
# # 1) INSTALL DEPENDENCIES
# ################################################################################
# !pip install -q ultralytics torch torchvision  # Ensure torch/torchvision, YOLO installed

# ################################################################################
# # 2) DOWNLOAD REAL IMAGES FROM THE INTERNET
# #    We'll use a few example images from the YOLOv5 repo + other open sources.
# ################################################################################
# !pip install ultralytics torch torchvision

# import os
# import requests
# from PIL import Image
# import time
# import torch
# import torchvision
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np

# # If you want to confirm file removal:
# !rm -f test_image_0.jpg test_image_1.jpg test_image_2.jpg test_image_3.jpg test_image_4.jpg



# image_urls = [
#     "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bus.jpg",
#     "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/dog.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/cat.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/person.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/traffic.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/giraffe.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/horse.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bird.jpg",
#     # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bicycle.jpg"
# ]


# image_files = []

# # Force re-download each file
# for idx, url in enumerate(image_urls):
#     out_name = f"test_image_{idx}.jpg"
#     print(f"Downloading: {url} -> {out_name}")
#     !wget {url} -O {out_name}  # remove -q to see output
#     image_files.append(out_name)

# print("\nCheck files:")
# !ls -lh test_image_*.jpg

# # Quick check: open each image in Python
# for f in image_files:
#     try:
#         im = Image.open(f)
#         im.verify()  # attempt to identify the file
#         print(f"{f} OK")
#     except Exception as e:
#         print(f"{f} is corrupted or not an image. Error: {e}")



# # import os

# # image_urls = [
# #     "https://ultralytics.com/images/bus.jpg",
# #     "https://ultralytics.com/images/zidane.jpg",
# #     "https://ultralytics.com/images/bicycle.jpg",
# #     "https://ultralytics.com/images/traffic.jpg",
# #     "https://ultralytics.com/images/bird.jpg"
# # ]
# # image_files = []

# # for idx, url in enumerate(image_urls):
# #     out_name = f"test_image_{idx}.jpg"
# #     # Force re-download every time
# #     !wget -q {url} -O {out_name}
# #     image_files.append(out_name)


# ################################################################################
# # 3) IMPORT LIBRARIES & LOAD MODELS
# ################################################################################
# import time
# import torch
# import torchvision
# from PIL import Image
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

# # YOLOv8 from ultralytics
# from ultralytics import YOLO

# # TorchVision detection models
# # NOTE: "DEFAULT" weights correspond to COCO-pretrained.
# fasterrcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT").eval()
# ssd = torchvision.models.detection.ssdlite320_mobilenet_v3_large(weights="DEFAULT").eval()
# retinanet = torchvision.models.detection.retinanet_resnet50_fpn(weights="DEFAULT").eval()
# maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT").eval()

# # YOLOv8 (smallest or 'n' version for speed; you can pick others, e.g. 'yolov8s.pt')
# yolo_model = YOLO('yolov8n.pt')  # automatically downloads if not present

# # Put models into a dictionary for iteration
# models_to_evaluate = {
#     "YOLOv8": yolo_model,          # via ultralytics
#     "FasterRCNN": fasterrcnn,      # via TorchVision
#     "SSD": ssd,                    # via TorchVision
#     "RetinaNet": retinanet,        # via TorchVision
#     "MaskRCNN": maskrcnn           # via TorchVision
# }

# ################################################################################
# # 4) DEFINE INFERENCE FUNCTIONS FOR YOLO & TORCHVISION
# ################################################################################

# # For TorchVision detection models
# def run_torchvision_inference(model, images):
#     """Runs inference on a list of PIL images, returns predictions and total time."""
#     transform = torchvision.transforms.ToTensor()
#     input_tensors = [transform(img) for img in images]

#     start_time = time.time()
#     with torch.no_grad():
#         preds = model(input_tensors)
#     total_time = time.time() - start_time

#     return preds, total_time

# # For YOLOv8 model (ultralytics)
# def run_yolov8_inference(model, images):
#     """
#     Runs inference using ultralytics YOLO model.
#     Returns predictions and total time.
#     """
#     start_time = time.time()
#     # YOLO can handle file paths or PIL/numpy images, but let's pass them as PIL or np arrays.
#     # We'll collect results for all images in a single list call
#     preds = model.predict(images, verbose=False)  # pass a list of images
#     total_time = time.time() - start_time
#     return preds, total_time

# ################################################################################
# # 5) RUN INFERENCE ON ALL MODELS
# #    We'll store for each model:
# #       - total inference time (for 5 images)
# #       - # of detections in the first image
# ################################################################################
# results_table = []

# # Load the 5 images into memory as PIL
# test_images = [Image.open(f).convert("RGB") for f in image_files]

# for model_name, model_obj in models_to_evaluate.items():
#     if model_name == "YOLOv8":
#         # YOLO Inference
#         preds, inf_time = run_yolov8_inference(model_obj, test_images)
#         # preds is a list of ultralytics.yolo.engine.results.Results objects
#         # Let's get the number of boxes from the first image:
#         # "boxes" is a tensor of shape Nx6 [x1, y1, x2, y2, score, class]
#         if len(preds) > 0:
#             first_img_boxes = preds[0].boxes
#             detection_count = len(first_img_boxes) if first_img_boxes is not None else 0
#         else:
#             detection_count = 0
#     else:
#         # TorchVision Inference
#         preds, inf_time = run_torchvision_inference(model_obj, test_images)
#         # Typically preds[i]["boxes"] is Nx4, so detection_count is N
#         if len(preds) > 0 and "boxes" in preds[0]:
#             detection_count = len(preds[0]["boxes"])
#         else:
#             detection_count = 0

#     results_table.append({
#         "Model": model_name,
#         "Inference Time (sec, 5 imgs)": f"{inf_time:.3f}",
#         "Objects Detected (1st Img)": detection_count
#     })

# df_results = pd.DataFrame(results_table)
# print("=== INFERENCE RESULTS SUMMARY ===")
# print(df_results)

# ################################################################################
# # 6) VISUALIZE BOUNDING BOXES ON A SAMPLE IMAGE
# #    We'll display separate plots, one for each model, for the first test image.
# ################################################################################
# sample_image = test_images[0]  # let's pick the first image in the list

# def draw_boxes_torchvision(img, pred, score_thresh=0.5):
#     """
#     For TorchVision outputs:
#     - pred is a dict containing "boxes", "scores", ...
#     - We'll draw boxes for scores >= threshold.
#     """
#     boxes = pred["boxes"]
#     scores = pred["scores"]

#     np_img = np.array(img)
#     fig, ax = plt.subplots()
#     ax.imshow(np_img)

#     for box, sc in zip(boxes, scores):
#         if sc >= score_thresh:
#             x1, y1, x2, y2 = box
#             w, h = x2 - x1, y2 - y1
#             rect = plt.Rectangle((x1, y1), w, h, fill=False, linewidth=2)
#             ax.add_patch(rect)
#     ax.set_title("TorchVision Predictions")
#     plt.show()

# def draw_boxes_yolov8(img, yolo_result, score_thresh=0.5):
#     import matplotlib.patches as patches
#     import numpy as np
#     import matplotlib.pyplot as plt

#     np_img = np.array(img)
#     fig, ax = plt.subplots()
#     ax.imshow(np_img)

#     if yolo_result.boxes is not None:
#         for row in yolo_result.boxes.data:
#             # Move row to CPU and convert to NumPy
#             row = row.cpu().numpy()
#             x1, y1, x2, y2, s, cls = row

#             if s >= score_thresh:
#                 w, h = x2 - x1, y2 - y1
#                 rect = patches.Rectangle(
#                     (x1, y1), w, h,
#                     fill=False, linewidth=2
#                 )
#                 ax.add_patch(rect)

#     ax.set_title("YOLOv8 Predictions")
#     plt.show()


# print("\n=== SAMPLE VISUALIZATIONS ON FIRST IMAGE ===")
# for model_name, model_obj in models_to_evaluate.items():
#     print(f"Showing predictions for: {model_name}")
#     if model_name == "YOLOv8":
#         preds, _ = run_yolov8_inference(model_obj, [sample_image])
#         draw_boxes_yolov8(sample_image, preds[0], score_thresh=0.5)
#     else:
#         preds, _ = run_torchvision_inference(model_obj, [sample_image])
#         draw_boxes_torchvision(sample_image, preds[0], score_thresh=0.5)


In [5]:
import sys
import subprocess
import pkg_resources

def install_package(package, version=None):
    """Install a package if not already installed, with optional version."""
    try:
        pkg = f"{package}=={version}" if version else package
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print(f"Successfully installed {pkg}")
    except Exception as e:
        print(f"Error installing {package}: {e}")

# Core dependencies
install_package("ultralytics")  # For YOLOv8
install_package("torch")
install_package("torchvision")
install_package("matplotlib")
install_package("pandas")
install_package("Pillow")
install_package("requests")
install_package("numpy")
install_package("timm")  # For EfficientDet

# Additional dependencies for other models
install_package("segmentation-models-pytorch")  # For U-Net
install_package("timm")  # For advanced models
install_package("transformers")  # For DETR

# Install PyTorch3D for YOLACT
try:
    import torch
    if torch.cuda.is_available():
        install_package("pytorch3d")
except:
    print("Skipping PyTorch3D installation - requires CUDA")

import os
import requests
from PIL import Image
import time
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from pathlib import Path

# Create a directory for test images
os.makedirs("test_images", exist_ok=True)

# Define a more diverse set of images (10 images as requested)
image_urls = [
    "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bus.jpg",
    "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/dog.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/cat.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/traffic.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/horses.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/motorcycle.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/person.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bird.jpg",
    # "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bicycle.jpg"
]

image_files = []

# Download images
for idx, url in enumerate(image_urls):
    out_name = f"test_images/image_{idx}.jpg"
    print(f"Downloading: {url} -> {out_name}")

    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(out_name, 'wb') as f:
                f.write(response.content)

            # Verify the image
            try:
                img = Image.open(out_name)
                img.verify()
                image_files.append(out_name)
                print(f"{out_name} successfully downloaded and verified.")
            except Exception as e:
                print(f"Error with {out_name}: {e}")
        else:
            print(f"Failed to download {url}: {response.status_code}")

            # Fallback to wget
            os.system(f"wget {url} -O {out_name}")

            # Verify after wget
            try:
                img = Image.open(out_name)
                img.verify()
                image_files.append(out_name)
                print(f"{out_name} successfully downloaded with wget and verified.")
            except Exception as e:
                print(f"Error with {out_name} after wget: {e}")

    except Exception as e:
        print(f"Error downloading {url}: {e}")

print(f"\nSuccessfully downloaded {len(image_files)} images")

import time
import torch
import torchvision
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set device for PyTorch models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

models_to_evaluate = {}

# YOLOv8 models (multiple sizes)
try:
    from ultralytics import YOLO
    yolo_sizes = ['n', 's', 'm', 'l']  # nano, small, medium, large
    for size in yolo_sizes:
        model_name = f"YOLOv8-{size}"
        try:
            models_to_evaluate[model_name] = YOLO(f'yolov8{size}.pt')
            print(f"Loaded {model_name}")
        except Exception as e:
            print(f"Error loading {model_name}: {e}")
except Exception as e:
    print(f"Error importing YOLO: {e}")

# YOLOv5 (Legacy)
try:
    # Legacy YOLOv5 requires different import strategy
    # We'll use the Hub model directly
    import torch
    yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    models_to_evaluate["YOLOv5"] = yolov5
    print("Loaded YOLOv5")
except Exception as e:
    print(f"Error loading YOLOv5: {e}")

# TorchVision models
try:
    models_to_evaluate["FasterRCNN"] = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT").to(device).eval()
    models_to_evaluate["SSD"] = torchvision.models.detection.ssdlite320_mobilenet_v3_large(weights="DEFAULT").to(device).eval()
    models_to_evaluate["RetinaNet"] = torchvision.models.detection.retinanet_resnet50_fpn(weights="DEFAULT").to(device).eval()
    models_to_evaluate["MaskRCNN"] = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT").to(device).eval()
    print("Loaded TorchVision models")
except Exception as e:
    print(f"Error loading TorchVision models: {e}")

# DETR (DEtection TRansformer) from Facebook Research
try:
    detr = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
    detr.to(device).eval()
    models_to_evaluate["DETR"] = detr
    print("Loaded DETR")
except Exception as e:
    print(f"Error loading DETR: {e}")

# U-Net for Segmentation
try:
    import segmentation_models_pytorch as smp
    unet_model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    ).to(device).eval()
    models_to_evaluate["U-Net"] = unet_model
    print("Loaded U-Net")
except Exception as e:
    print(f"Error loading U-Net: {e}")

# EfficientDet (from timm)
try:
    import timm
    efficientdet = torch.hub.load('rwightman/gen-efficientdet-pytorch', 'tf_efficientdet_d0', pretrained=True)
    efficientdet.to(device).eval()
    models_to_evaluate["EfficientDet"] = efficientdet
    print("Loaded EfficientDet")
except Exception as e:
    print(f"Error loading EfficientDet: {e}")

# FCOS (Fully Convolutional One-Stage Object Detection)
try:
    fcos = torch.hub.load('tianzhi0549/FCOS', 'fcos_r50_fpn_1x', pretrained=True)
    fcos.to(device).eval()
    models_to_evaluate["FCOS"] = fcos
    print("Loaded FCOS")
except Exception as e:
    print(f"Error loading FCOS: {e}")

# YOLACT (You Only Look At CoefficienTs)
try:
    yolact = torch.hub.load('dbolya/yolact', 'yolact_resnet50_54_800000', pretrained=True)
    yolact.to(device).eval()
    models_to_evaluate["YOLACT"] = yolact
    print("Loaded YOLACT")
except Exception as e:
    print(f"Error loading YOLACT: {e}")

print(f"\nLoaded {len(models_to_evaluate)} models for evaluation.")

# Generic function to preprocess images
def preprocess_image(image, target_size=None):
    """Preprocess PIL image for model inference."""
    # Convert PIL Image to RGB if needed
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Resize if target_size is provided
    if target_size:
        image = image.resize(target_size)

    return image

# For TorchVision detection models
def run_torchvision_inference(model, images):
    """Runs inference with TorchVision models."""
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
    ])

    input_tensors = [transform(img).to(device) for img in images]

    start_time = time.time()
    with torch.no_grad():
        predictions = model(input_tensors)
    total_time = time.time() - start_time

    return predictions, total_time

# For YOLOv8 models
def run_yolov8_inference(model, images):
    """Runs inference with YOLOv8 models."""
    start_time = time.time()
    predictions = model.predict(images, verbose=False)
    total_time = time.time() - start_time

    return predictions, total_time

# For YOLOv5 models
def run_yolov5_inference(model, images):
    """Runs inference with YOLOv5 models."""
    # Convert PIL images to paths or keep as PIL
    start_time = time.time()
    predictions = model(images)
    total_time = time.time() - start_time

    return predictions, total_time

# For DETR model
def run_detr_inference(model, images):
    """Runs inference with DETR model."""
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    input_tensors = [transform(img).unsqueeze(0).to(device) for img in images]

    start_time = time.time()
    with torch.no_grad():
        predictions = []
        for tensor in input_tensors:
            output = model(tensor)
            predictions.append({
                'boxes': output['pred_boxes'][0].cpu(),
                'scores': output['pred_logits'][0].softmax(-1)[..., :-1].max(-1)[0].cpu(),
                'labels': output['pred_logits'][0].softmax(-1)[..., :-1].max(-1)[1].cpu()
            })
    total_time = time.time() - start_time

    return predictions, total_time

# For U-Net segmentation model
def run_unet_inference(model, images):
    """Runs inference with U-Net segmentation model."""
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        torchvision.transforms.Resize((256, 256))
    ])

    input_tensors = [transform(img).unsqueeze(0).to(device) for img in images]

    start_time = time.time()
    with torch.no_grad():
        predictions = []
        for tensor in input_tensors:
            output = model(tensor)
            # For semantic segmentation, we count segments differently
            mask = (output > 0.5).float().cpu()
            predictions.append({'masks': mask})
    total_time = time.time() - start_time

    return predictions, total_time

# For EfficientDet
def run_efficientdet_inference(model, images):
    """Runs inference with EfficientDet model."""
    # EfficientDet uses its own preprocessing
    start_time = time.time()
    with torch.no_grad():
        predictions = []
        for img in images:
            img_tensor = torchvision.transforms.ToTensor()(img).unsqueeze(0).to(device)
            output = model(img_tensor)
            predictions.append({
                'boxes': output[0]['boxes'],
                'scores': output[0]['scores'],
                'labels': output[0]['labels']
            })
    total_time = time.time() - start_time

    return predictions, total_time

# For FCOS
def run_fcos_inference(model, images):
    """Runs inference with FCOS model."""
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    input_tensors = [transform(img).unsqueeze(0).to(device) for img in images]

    start_time = time.time()
    with torch.no_grad():
        predictions = []
        for tensor in input_tensors:
            output = model(tensor)
            predictions.append({
                'boxes': output[0]['boxes'].cpu(),
                'scores': output[0]['scores'].cpu(),
                'labels': output[0]['labels'].cpu()
            })
    total_time = time.time() - start_time

    return predictions, total_time

# For YOLACT
def run_yolact_inference(model, images):
    """Runs inference with YOLACT model."""
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    start_time = time.time()
    with torch.no_grad():
        predictions = []
        for img in images:
            img_tensor = transform(img).unsqueeze(0).to(device)
            classes, scores, boxes, masks = model(img_tensor)
            predictions.append({
                'boxes': boxes[0].cpu() if len(boxes) > 0 else torch.tensor([]),
                'scores': scores[0].cpu() if len(scores) > 0 else torch.tensor([]),
                'labels': classes[0].cpu() if len(classes) > 0 else torch.tensor([]),
                'masks': masks[0].cpu() if len(masks) > 0 else torch.tensor([])
            })
    total_time = time.time() - start_time

    return predictions, total_time


# Load all images as PIL objects
test_images = [Image.open(f).convert("RGB") for f in image_files]

results_table = []
all_predictions = {}  # Store predictions for visualization

for model_name, model in models_to_evaluate.items():
    print(f"Running inference with {model_name}...")

    try:
        # Different handling based on model type
        if "YOLOv8" in model_name:
            predictions, inf_time = run_yolov8_inference(model, test_images)
            detection_count = len(predictions[0].boxes) if len(predictions) > 0 and predictions[0].boxes is not None else 0
        elif model_name == "YOLOv5":
            predictions, inf_time = run_yolov5_inference(model, test_images)
            detection_count = len(predictions.xyxy[0]) if len(predictions.xyxy) > 0 else 0
        elif model_name in ["FasterRCNN", "SSD", "RetinaNet", "MaskRCNN"]:
            predictions, inf_time = run_torchvision_inference(model, test_images)
            detection_count = len(predictions[0]["boxes"]) if len(predictions) > 0 and "boxes" in predictions[0] else 0
        elif model_name == "DETR":
            predictions, inf_time = run_detr_inference(model, test_images)
            detection_count = len(predictions[0]["boxes"]) if len(predictions) > 0 and "boxes" in predictions[0] else 0
        elif model_name == "U-Net":
            predictions, inf_time = run_unet_inference(model, test_images)
            # For U-Net, we count segments differently
            detection_count = torch.sum(predictions[0]['masks'] > 0.5).item() if len(predictions) > 0 else 0
        elif model_name == "EfficientDet":
            predictions, inf_time = run_efficientdet_inference(model, test_images)
            detection_count = len(predictions[0]["boxes"]) if len(predictions) > 0 and "boxes" in predictions[0] else 0
        elif model_name == "FCOS":
            predictions, inf_time = run_fcos_inference(model, test_images)
            detection_count = len(predictions[0]["boxes"]) if len(predictions) > 0 and "boxes" in predictions[0] else 0
        elif model_name == "YOLACT":
            predictions, inf_time = run_yolact_inference(model, test_images)
            detection_count = len(predictions[0]["boxes"]) if len(predictions) > 0 and "boxes" in predictions[0] else 0
        else:
            # Generic fallback
            print(f"Warning: No specific handling for {model_name}, skipping")
            continue

        # Store predictions for visualization
        all_predictions[model_name] = predictions

        # Add to results table
        fps = len(test_images) / inf_time
        results_table.append({
            "Model": model_name,
            "Inference Time (sec)": f"{inf_time:.3f}",
            "FPS": f"{fps:.2f}",
            "Objects Detected (1st Img)": detection_count
        })

        print(f"  -> {model_name}: {inf_time:.3f} sec, {detection_count} objects detected")

    except Exception as e:
        print(f"Error running inference with {model_name}: {e}")

# Create DataFrame with results
df_results = pd.DataFrame(results_table)

# Sort by inference time
df_results = df_results.sort_values("Inference Time (sec)")

print("\n=== PERFORMANCE COMPARISON ===")
print(df_results)

# Save results to CSV
df_results.to_csv("model_comparison_results.csv", index=False)
print("Results saved to model_comparison_results.csv")


def visualize_detections(image, predictions, model_name, threshold=0.3):
    """Visualize detections for different model types."""

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(np.array(image))

    # Different handling based on model type
    if "YOLOv8" in model_name:
        # YOLOv8 predictions
        if predictions.boxes is not None:
            for box in predictions.boxes.data:
                box = box.cpu().numpy()
                x1, y1, x2, y2, score, class_id = box
                if score >= threshold:
                    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                        fill=False, edgecolor='red', linewidth=2)
                    ax.add_patch(rect)
                    class_name = predictions.names[int(class_id)]
                    ax.text(x1, y1, f"{class_name}: {score:.2f}",
                            bbox=dict(facecolor='yellow', alpha=0.5))

    elif model_name == "YOLOv5":
        # YOLOv5 predictions
        boxes = predictions.xyxy[0]
        for box in boxes:
            box = box.cpu().numpy()
            x1, y1, x2, y2, score, class_id = box
            if score >= threshold:
                rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                    fill=False, edgecolor='red', linewidth=2)
                ax.add_patch(rect)
                class_name = predictions.names[int(class_id)]
                ax.text(x1, y1, f"{class_name}: {score:.2f}",
                        bbox=dict(facecolor='yellow', alpha=0.5))

    elif model_name in ["FasterRCNN", "SSD", "RetinaNet", "MaskRCNN", "DETR", "EfficientDet", "FCOS", "YOLACT"]:
        # TorchVision-style predictions
        if "boxes" in predictions and len(predictions["boxes"]) > 0:
            boxes = predictions["boxes"].cpu().numpy()
            scores = predictions["scores"].cpu().numpy()
            labels = predictions["labels"].cpu().numpy() if "labels" in predictions else None

            # If MaskRCNN, also display masks
            if "masks" in predictions and model_name == "MaskRCNN":
                masks = predictions["masks"].cpu().numpy()

                for i, (box, score) in enumerate(zip(boxes, scores)):
                    if score >= threshold:
                        x1, y1, x2, y2 = box
                        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                           fill=False, edgecolor='red', linewidth=2)
                        ax.add_patch(rect)

                        # Display label with score
                        label_text = f"Class {labels[i]}: {score:.2f}" if labels is not None else f"Score: {score:.2f}"
                        ax.text(x1, y1, label_text, bbox=dict(facecolor='yellow', alpha=0.5))

                        # Add mask if available
                        if i < len(masks):
                            mask = masks[i][0]
                            mask = mask > 0.5  # Threshold
                            masked = np.ma.masked_where(~mask, mask)
                            ax.imshow(masked, alpha=0.5, cmap='jet')
            else:
                # Standard bounding box visualization
                for i, (box, score) in enumerate(zip(boxes, scores)):
                    if score >= threshold:
                        x1, y1, x2, y2 = box
                        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                           fill=False, edgecolor='red', linewidth=2)
                        ax.add_patch(rect)

                        # Display label with score
                        label_text = f"Class {labels[i]}: {score:.2f}" if labels is not None else f"Score: {score:.2f}"
                        ax.text(x1, y1, label_text, bbox=dict(facecolor='yellow', alpha=0.5))

    elif model_name == "U-Net":
        # For U-Net segmentation masks
        if "masks" in predictions:
            mask = predictions["masks"].squeeze().cpu().numpy()
            mask = mask > 0.5  # Apply threshold
            masked = np.ma.masked_where(~mask, mask)
            ax.imshow(masked, alpha=0.5, cmap='jet')

    ax.set_title(f"{model_name} Predictions")
    plt.tight_layout()
    plt.savefig(f"detections_{model_name.replace('-', '_')}.png")
    plt.close()


# Sample image for visualization (first image)
sample_image = test_images[0]

# Visualize predictions from each model
print("\n=== GENERATING VISUALIZATIONS ===")

for model_name, predictions in all_predictions.items():
    try:
        print(f"Visualizing {model_name} predictions...")

        if "YOLOv8" in model_name:
            visualize_detections(sample_image, predictions[0], model_name)
        elif model_name == "YOLOv5":
            visualize_detections(sample_image, predictions, model_name)
        else:
            visualize_detections(sample_image, predictions[0], model_name)

        print(f"  -> Saved visualization for {model_name}")
    except Exception as e:
        print(f"Error visualizing {model_name}: {e}")


def create_grid_visualization():
    """Create a grid visualization of all model detections."""
    # Collect all saved visualization images
    visualization_files = [f for f in os.listdir() if f.startswith('detections_') and f.endswith('.png')]

    if len(visualization_files) == 0:
        print("No visualization files found.")
        return

    # Calculate grid dimensions
    n_images = len(visualization_files)
    n_cols = min(3, n_images)
    n_rows = math.ceil(n_images / n_cols)

    # Create a grid figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))

    # Flatten axes if needed
    if n_rows > 1 or n_cols > 1:
        axes = axes.flatten()
    else:
        axes = [axes]

    # Add each visualization to the grid
    for i, file in enumerate(visualization_files):
        if i < len(axes):
            # Load image
            img = plt.imread(file)
            axes[i].imshow(img)

            # Extract model name from filename
            model_name = file.replace('detections_', '').replace('.png', '').replace('_', '-')
            axes[i].set_title(model_name)
            axes[i].axis('off')

    # Hide any unused axes
    for i in range(len(visualization_files), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.savefig("all_models_comparison.png", dpi=300)
    plt.close()

    print("Created grid visualization of all models: all_models_comparison.png")

create_grid_visualization()


Error installing pytorch3d: Command '['/usr/bin/python3', '-m', 'pip', 'install', 'pytorch3d']' returned non-zero exit status 1.
Downloading: https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bus.jpg -> test_images/image_0.jpg
test_images/image_0.jpg successfully downloaded and verified.
Downloading: https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg -> test_images/image_1.jpg
test_images/image_1.jpg successfully downloaded and verified.

Successfully downloaded 2 images
Using device: cuda
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.8MB/s]

Loaded YOLOv8-n



100%|██████████| 21.5M/21.5M [00:00<00:00, 138MB/s] 


Loaded YOLOv8-s


...

Loaded U-Net
Error loading EfficientDet: HTTP Error 404: Not Found


Downloading: "https://github.com/tianzhi0549/FCOS/zipball/master" to /root/.cache/torch/hub/master.zip


Error loading FCOS: [Errno 2] No such file or directory: '/root/.cache/torch/hub/tianzhi0549_FCOS_master/hubconf.py'


Downloading: "https://github.com/dbolya/yolact/zipball/master" to /root/.cache/torch/hub/master.zip


Error loading YOLACT: [Errno 2] No such file or directory: '/root/.cache/torch/hub/dbolya_yolact_master/hubconf.py'

Loaded 10 models for evaluation.
Running inference with YOLOv8-n...
  -> YOLOv8-n: 0.638 sec, 5 objects detected
Running inference with YOLOv8-s...
  -> YOLOv8-s: 0.291 sec, 5 objects detected
Running inference with YOLOv8-m...
  -> YOLOv8-m: 0.449 sec, 5 objects detected
Running inference with YOLOv8-l...
  -> YOLOv8-l: 0.605 sec, 6 objects detected
Running inference with YOLOv5...
  -> YOLOv5: 0.029 sec, 5 objects detected
Running inference with FasterRCNN...
  -> FasterRCNN: 0.350 sec, 31 objects detected
Running inference with SSD...
  -> SSD: 0.256 sec, 300 objects detected
Running inference with RetinaNet...
  -> RetinaNet: 0.124 sec, 273 objects detected
Running inference with MaskRCNN...
  -> MaskRCNN: 0.103 sec, 32 objects detected
Running inference with U-Net...
  -> U-Net: 0.039 sec, 91 objects detected

=== PERFORMANCE COMPARISON ===
        Model Inference T

## **Task 5-2**

In [3]:
# import os
# import time
# import requests
# import zipfile
# import random
# from io import BytesIO
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# import torchvision.models as models
# from torchvision.utils import save_image
# from tqdm import tqdm

# # Set device to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Create directories for datasets and results
# os.makedirs("data", exist_ok=True)
# os.makedirs("data/content", exist_ok=True)
# os.makedirs("data/style", exist_ok=True)
# os.makedirs("results", exist_ok=True)

# def download_and_extract_zip(url, extract_to):
#     """Download and extract a zip file from URL to the specified directory."""
#     try:
#         print(f"Downloading from {url}...")
#         response = requests.get(url, stream=True)
#         z = zipfile.ZipFile(BytesIO(response.content))
#         print("Extracting zip file...")
#         z.extractall(extract_to)
#         print(f"Files extracted to {extract_to}")
#         return True
#     except Exception as e:
#         print(f"Error downloading/extracting: {e}")
#         return False

# # Download sample content images (COCO subset)
# content_url = "https://github.com/pytorch/examples/raw/main/fast_neural_style/images.zip"
# download_and_extract_zip(content_url, "data/content")

# # Instead of downloading the full WikiArt dataset (which is large),
# # we'll download a few famous paintings for style images
# style_images = [
#     ("starry_night.jpg", "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"),
#     ("the_scream.jpg", "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg/800px-Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg"),
#     ("picasso.jpg", "https://upload.wikimedia.org/wikipedia/en/thumb/9/9c/Pablo_Picasso%2C_1910%2C_Girl_with_a_Mandolin_%28Fanny_Tellier%29%2C_oil_on_canvas%2C_100.3_x_73.6_cm%2C_Museum_of_Modern_Art_New_York..jpg/800px-Pablo_Picasso%2C_1910%2C_Girl_with_a_Mandolin_%28Fanny_Tellier%29%2C_oil_on_canvas%2C_100.3_x_73.6_cm%2C_Museum_of_Modern_Art_New_York..jpg"),
#     ("mona_lisa.jpg", "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/800px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"),
#     ("kandinsky.jpg", "https://upload.wikimedia.org/wikipedia/commons/a/ad/Wassily_Kandinsky_Composition_VIII.jpg"),
#     ("great_wave.jpg", "https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg"),
#     ("candy.jpg", "https://github.com/pytorch/examples/blob/main/fast_neural_style/images/style-images/candy.jpg"),



# ]

# for filename, url in style_images:
#     try:
#         print(f"Downloading {filename}...")
#         response = requests.get(url)
#         img_path = os.path.join("data/style", filename)
#         with open(img_path, "wb") as f:
#             f.write(response.content)
#         print(f"Saved to {img_path}")

#         # Verify the image
#         try:
#             Image.open(img_path).verify()
#         except:
#             print(f"Warning: {filename} may be corrupted")
#     except Exception as e:
#         print(f"Error downloading {filename}: {e}")

# # List the downloaded images
# print("\nContent images:")
# content_images = [f for f in os.listdir("data/content") if f.endswith(('.jpg', '.jpeg', '.png'))]
# print(content_images[:5])  # Show first 5

# print("\nStyle images:")
# style_images = [f for f in os.listdir("data/style") if f.endswith(('.jpg', '.jpeg', '.png'))]
# print(style_images)


# # Define image loading and preprocessing
# # def load_image(image_path, max_size=512):
# #     """Load an image, resize it and convert to tensor."""
# #     image = Image.open(image_path).convert('RGB')

# #     # Resize while keeping aspect ratio
# #     if max(image.size) > max_size:
# #         ratio = max_size / max(image.size)
# #         new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
# #         image = image.resize(new_size, Image.LANCZOS)

# #     # Convert to tensor
# #     transform = transforms.Compose([
# #         transforms.ToTensor(),
# #         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# #     ])

# #     # Add batch dimension
# #     image = transform(image).unsqueeze(0)

# #     return image.to(device)

# def load_img(path_to_img, max_dim=512):
#     """Load and preprocess images to make them suitable for the model"""
#     # Handle different input types
#     if isinstance(path_to_img, str):
#         if path_to_img.startswith('http'):
#             # Download image from URL using requests with a proper User-Agent
#             headers = {
#                 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#             }
#             try:
#                 response = requests.get(path_to_img, headers=headers, timeout=10)
#                 response.raise_for_status()  # Check if the request was successful

#                 # Use PIL to open the image
#                 img = Image.open(BytesIO(response.content))
#                 # Convert to numpy array and then to tensor
#                 img = np.array(img)
#                 img = torch.from_numpy(img).float()
#             except Exception as e:
#                 print(f"Error downloading or processing image from {path_to_img}: {str(e)}")
#                 raise
#         else:
#             try:
#                 img = Image.open(path_to_img)
#                 img = np.array(img)
#                 img = torch.from_numpy(img).float()
#             except Exception as e:
#                 print(f"Error loading local image {path_to_img}: {str(e)}")
#                 raise
#     else:
#         img = path_to_img  # Assume it's already a tensor

#     # Ensure the image has 3 channels (RGB)
#     if img.shape[-1] == 4:  # If RGBA (with alpha channel)
#         img = img[..., :3]  # Keep only RGB

#     # Convert to float in [0, 1] range
#     if img.max() > 1.0:
#         img = img / 255.0

#     # Resize the image while preserving aspect ratio
#     height, width = img.shape[0], img.shape[1]
#     long_dim = max(height, width)
#     scale = max_dim / long_dim
#     new_height = int(height * scale)
#     new_width = int(width * scale)

#     # Reshape and normalize
#     img = img.permute(2, 0, 1)  # Convert to [C, H, W]
#     img = F.interpolate(img.unsqueeze(0), size=(new_height, new_width), mode='bilinear', align_corners=False).squeeze(0)

#     # Normalize using ImageNet stats
#     mean = torch.tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
#     std = torch.tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
#     img = (img - mean) / std

#     # Add batch dimension
#     img = img.unsqueeze(0)

#     return img.to(device)

# def tensor_to_image(tensor):
#     """Convert tensor to PIL Image."""
#     # Clone the tensor to avoid modifying the original
#     tensor = tensor.clone().detach().cpu()

#     # Denormalize
#     tensor = tensor * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)

#     # Clamp to [0, 1]
#     tensor = torch.clamp(tensor, 0, 1)

#     # Convert to PIL image
#     tensor = tensor.squeeze().permute(1, 2, 0).numpy() * 255
#     return Image.fromarray(tensor.astype('uint8'))

# # Select random content and style images
# def get_random_images(num_content=1, num_style=1):
#     """Select random content and style images."""
#     content_paths = [os.path.join("data/content", img) for img in
#                      random.sample(content_images, min(num_content, len(content_images)))]

#     style_paths = [os.path.join("data/style", img) for img in
#                    random.sample(style_images, min(num_style, len(style_images)))]

#     return content_paths, style_paths

# # VGG19 feature extraction
# class VGGFeatures(nn.Module):
#     def __init__(self):
#         super(VGGFeatures, self).__init__()
#         # Load pretrained VGG19 and extract specific layers
#         vgg = models.vgg19(pretrained=True).features.to(device).eval()

#         # Freeze parameters
#         for param in vgg.parameters():
#             param.requires_grad = False

#         # Feature layers we're interested in
#         self.layers = {
#             '0': 'conv1_1',
#             '5': 'conv2_1',
#             '10': 'conv3_1',
#             '19': 'conv4_1',
#             '21': 'conv4_2',  # content representation
#             '28': 'conv5_1'
#         }

#         self.model = vgg

#     def forward(self, x):
#         features = {}
#         for name, layer in self.model._modules.items():
#             x = layer(x)
#             if name in self.layers:
#                 features[self.layers[name]] = x

#         return features

# # Gram Matrix for style representation
# def gram_matrix(tensor):
#     """Calculate Gram Matrix for style representation."""
#     batch, channel, height, width = tensor.size()
#     tensor = tensor.view(batch, channel, height * width)

#     # Compute gram matrix
#     gram = torch.bmm(tensor, tensor.transpose(1, 2))

#     # Normalize by total elements
#     return gram.div(channel * height * width)

# def classic_style_transfer(content_img, style_img, num_steps=300, style_weight=1e6, content_weight=1):
#     """Implement the classic Neural Style Transfer algorithm by Gatys et al."""

#     print("\nRunning Classic Neural Style Transfer (Gatys et al.)...")

#     # Initialize VGG model for feature extraction
#     vgg = VGGFeatures().to(device)

#     # Create a target image initialized with the content image
#     target = content_img.clone().requires_grad_(True).to(device)

#     # Optimizer
#     optimizer = optim.LBFGS([target])

#     # Extract features
#     content_features = vgg(content_img)
#     style_features = vgg(style_img)

#     # Calculate style gram matrices
#     style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

#     # Style and content layer weights
#     style_layer_weights = {'conv1_1': 1.0, 'conv2_1': 0.8, 'conv3_1': 0.6, 'conv4_1': 0.4, 'conv5_1': 0.2}
#     content_layer = 'conv4_2'

#     # Progress bar
#     progress_bar = tqdm(range(num_steps), desc="Style Transfer Progress")

#     # Style transfer loop
#     count = [0]  # For closure

#     def closure():
#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         target_features = vgg(target)

#         # Content loss
#         content_loss = F.mse_loss(target_features[content_layer], content_features[content_layer])

#         # Style loss
#         style_loss = 0
#         for layer in style_layer_weights:
#             target_feature = target_features[layer]
#             target_gram = gram_matrix(target_feature)
#             style_gram = style_grams[layer]
#             layer_style_loss = style_layer_weights[layer] * F.mse_loss(target_gram, style_gram)
#             style_loss += layer_style_loss

#         # Total loss
#         total_loss = content_weight * content_loss + style_weight * style_loss

#         # Backward pass
#         total_loss.backward()

#         # Update progress
#         count[0] += 1
#         if count[0] % 50 == 0 or count[0] == num_steps:
#             progress_bar.update(50 if count[0] != num_steps else num_steps % 50 or 50)
#             progress_bar.set_postfix(
#                 content_loss=f"{content_loss.item():.4f}",
#                 style_loss=f"{style_loss.item():.4f}",
#                 total_loss=f"{total_loss.item():.4f}"
#             )

#         return total_loss

#     # Run optimization
#     for _ in range(num_steps // 50):
#         optimizer.step(closure)

#     progress_bar.close()

#     # Convert result to image
#     result_img = tensor_to_image(target.detach())

#     return result_img


# # Define the Transformer Network for fast neural style transfer
# class TransformerNet(nn.Module):
#     def __init__(self):
#         super(TransformerNet, self).__init__()

#         # Initial convolution layers
#         self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
#         self.in1 = nn.InstanceNorm2d(32, affine=True)
#         self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
#         self.in2 = nn.InstanceNorm2d(64, affine=True)
#         self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
#         self.in3 = nn.InstanceNorm2d(128, affine=True)

#         # Residual blocks
#         self.res1 = ResidualBlock(128)
#         self.res2 = ResidualBlock(128)
#         self.res3 = ResidualBlock(128)
#         self.res4 = ResidualBlock(128)
#         self.res5 = ResidualBlock(128)

#         # Upsampling layers
#         self.deconv1 = UpsampleConvLayer(128, 64, kernel_size=3, stride=1, upsample=2)
#         self.in4 = nn.InstanceNorm2d(64, affine=True)
#         self.deconv2 = UpsampleConvLayer(64, 32, kernel_size=3, stride=1, upsample=2)
#         self.in5 = nn.InstanceNorm2d(32, affine=True)
#         self.deconv3 = ConvLayer(32, 3, kernel_size=9, stride=1)

#         # Non-linearities
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         y = self.relu(self.in1(self.conv1(x)))
#         y = self.relu(self.in2(self.conv2(y)))
#         y = self.relu(self.in3(self.conv3(y)))
#         y = self.res1(y)
#         y = self.res2(y)
#         y = self.res3(y)
#         y = self.res4(y)
#         y = self.res5(y)
#         y = self.relu(self.in4(self.deconv1(y)))
#         y = self.relu(self.in5(self.deconv2(y)))
#         y = self.deconv3(y)
#         return y

# class ConvLayer(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, stride):
#         super(ConvLayer, self).__init__()
#         padding = kernel_size // 2
#         self.reflection_pad = nn.ReflectionPad2d(padding)
#         self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

#     def forward(self, x):
#         out = self.reflection_pad(x)
#         out = self.conv2d(out)
#         return out

# class ResidualBlock(nn.Module):
#     def __init__(self, channels):
#         super(ResidualBlock, self).__init__()
#         self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
#         self.in1 = nn.InstanceNorm2d(channels, affine=True)
#         self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
#         self.in2 = nn.InstanceNorm2d(channels, affine=True)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         residual = x
#         out = self.relu(self.in1(self.conv1(x)))
#         out = self.in2(self.conv2(out))
#         out = out + residual
#         return out

# class UpsampleConvLayer(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
#         super(UpsampleConvLayer, self).__init__()
#         self.upsample = upsample
#         reflection_padding = kernel_size // 2
#         self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
#         self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

#     def forward(self, x):
#         x_in = x
#         if self.upsample:
#             x_in = F.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
#         out = self.reflection_pad(x_in)
#         out = self.conv2d(out)
#         return out

# def train_fast_style_transfer(style_img, dataset_path, epochs=2, batch_size=4):
#     """Train a Fast Neural Style Transfer model."""

#     print("\nTraining Fast Neural Style Transfer model (Johnson et al.)...")

#     # Define model and optimizer
#     transformer = TransformerNet().to(device)
#     optimizer = optim.Adam(transformer.parameters(), 1e-3)

#     # Load VGG for feature extraction
#     vgg = VGGFeatures().to(device)

#     # Extract style features
#     style_features = vgg(style_img)
#     style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

#     # Style and content layer weights
#     style_layer_weights = {'conv1_1': 1.0, 'conv2_1': 0.8, 'conv3_1': 0.6, 'conv4_1': 0.4, 'conv5_1': 0.2}
#     content_layer = 'conv4_2'

#     # Define content dataset
#     content_transform = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(256),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])

#     # Use a folder of images as the dataset
#     content_dataset = torchvision.datasets.ImageFolder(
#         root=os.path.dirname(dataset_path),
#         transform=content_transform
#     )

#     # Choose a subset for faster training
#     subset_indices = list(range(min(100, len(content_dataset))))
#     content_dataset = torch.utils.data.Subset(content_dataset, subset_indices)

#     # Create data loader
#     content_loader = torch.utils.data.DataLoader(
#         content_dataset, batch_size=batch_size, shuffle=True
#     )

#     # Define loss weights
#     style_weight = 1e5
#     content_weight = 1

#     # Training loop
#     for epoch in range(epochs):
#         transformer.train()
#         total_content_loss = 0
#         total_style_loss = 0
#         count = 0

#         # Progress bar
#         progress_bar = tqdm(content_loader, desc=f"Epoch {epoch+1}/{epochs}")

#         for batch_id, (x, _) in enumerate(progress_bar):
#             # Move batch to device
#             x = x.to(device)
#             count += len(x)

#             # Generate stylized images
#             y = transformer(x)

#             # Extract features
#             y_features = vgg(y)
#             x_features = vgg(x)

#             # Content loss
#             content_loss = F.mse_loss(y_features[content_layer], x_features[content_layer])

#             # Style loss
#             style_loss = 0
#             for layer in style_layer_weights:
#                 y_feature = y_features[layer]
#                 y_gram = gram_matrix(y_feature)
#                 style_gram = style_grams[layer].repeat(len(y), 1, 1)
#                 layer_style_loss = style_layer_weights[layer] * F.mse_loss(y_gram, style_gram)
#                 style_loss += layer_style_loss

#             # Total loss
#             loss = content_weight * content_loss + style_weight * style_loss

#             # Update
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Update statistics
#             total_content_loss += content_loss.item() * len(x)
#             total_style_loss += style_loss.item() * len(x)

#             # Update progress bar
#             progress_bar.set_postfix(
#                 content_loss=f"{total_content_loss/count:.4f}",
#                 style_loss=f"{total_style_loss/count:.4f}"
#             )

#     # Save the model
#     transformer.eval()
#     style_name = os.path.splitext(os.path.basename(style_img.name))[0]
#     model_path = f"fast_style_{style_name}.pth"
#     torch.save(transformer.state_dict(), model_path)
#     print(f"Model saved to {model_path}")

#     return transformer

# def apply_fast_style_transfer(content_img, transformer):
#     """Apply a trained Fast Neural Style Transfer model to an image."""

#     print("\nApplying Fast Neural Style Transfer...")

#     # Ensure model is in eval mode
#     transformer.eval()

#     # Process the image
#     with torch.no_grad():
#         output = transformer(content_img)

#     # Convert to image
#     result_img = tensor_to_image(output)

#     return result_img

# class AdaINStyleTransfer(nn.Module):
#     def __init__(self):
#         super(AdaINStyleTransfer, self).__init__()
#         # Load pretrained VGG19 encoder
#         vgg = models.vgg19(pretrained=True).features
#         self.encoder = nn.Sequential()

#         # Use layers up to relu4_1
#         for i in range(21):
#             self.encoder.add_module(str(i), vgg[i])

#         # Freeze encoder weights
#         for param in self.encoder.parameters():
#             param.requires_grad = False

#         # Define decoder (mirror of encoder)
#         self.decoder = nn.Sequential(
#             nn.Conv2d(512, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Upsample(scale_factor=2, mode='nearest'),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 128, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Upsample(scale_factor=2, mode='nearest'),
#             nn.Conv2d(128, 128, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(128, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Upsample(scale_factor=2, mode='nearest'),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, 3, kernel_size=3, padding=1),
#         )

#     def forward(self, content, style, alpha=1.0):
#         # Extract features
#         content_feat = self.encoder(content)
#         style_feat = self.encoder(style)

#         # Apply AdaIN transformation
#         t = self.adain(content_feat, style_feat)

#         # Adjust stylization strength
#         t = alpha * t + (1 - alpha) * content_feat

#         # Decode the features
#         output = self.decoder(t)

#         return output

#     def adain(self, content_feat, style_feat):
#         """Adaptive Instance Normalization."""
#         size = content_feat.size()

#         # Calculate mean and standard deviation of content and style features
#         content_mean, content_std = self.calc_mean_std(content_feat)
#         style_mean, style_std = self.calc_mean_std(style_feat)

#         # Normalize content feature
#         normalized_feat = (content_feat - content_mean.expand(size)) / content_std.expand(size)

#         # Scale and shift with style statistics
#         return normalized_feat * style_std.expand(size) + style_mean.expand(size)

#     def calc_mean_std(self, feat, eps=1e-5):
#         """Calculate mean and standard deviation of feature map."""
#         # Flatten [N, C, H, W] to [N, C, H*W]
#         size = feat.size()
#         feat_reshaped = feat.view(size[0], size[1], -1)

#         # Calculate mean and standard deviation
#         mean = feat_reshaped.mean(dim=2).view(size[0], size[1], 1, 1)
#         std = feat_reshaped.std(dim=2).view(size[0], size[1], 1, 1) + eps

#         return mean, std

# def adain_style_transfer(content_img, style_img, alpha=1.0):
#     """Apply AdaIN style transfer."""

#     print("\nRunning AdaIN Style Transfer...")

#     # Initialize model
#     adain_model = AdaINStyleTransfer().to(device)

#     # Ensure model is in eval mode
#     adain_model.eval()

#     # Process images
#     with torch.no_grad():
#         output = adain_model(content_img, style_img, alpha)

#     # Convert to image
#     result_img = tensor_to_image(output)

#     return result_img


# def load_pretrained_style_model(style_name):
#     """Load a pretrained style transfer model."""

#     # Define the model
#     style_model = TransformerNet().to(device)

#     # Try to download the model from torch hub
#     try:
#         # Check if the model exists locally first
#         model_path = f"pretrained_{style_name}.pth"

#         if os.path.exists(model_path):
#             # Load from local file
#             style_model.load_state_dict(torch.load(model_path))
#             print(f"Loaded pretrained style model from {model_path}")
#         else:
#             # Try to load from torch hub
#             model_url = f"https://github.com/rrmina/fast-neural-style-pytorch/tree/master/transforms/{style_name}.pth"

#             print(f"Downloading pretrained model from {model_url}")

#             response = requests.get(model_url)
#             if response.status_code == 200:
#                 # Save the model locally
#                 with open(model_path, "wb") as f:
#                     f.write(response.content)

#                 # Load the model
#                 style_model.load_state_dict(torch.load(model_path))
#                 print(f"Downloaded and loaded pretrained style model")
#             else:
#                 print(f"Failed to download pretrained model: {response.status_code}")
#                 return None

#     except Exception as e:
#         print(f"Error loading pretrained model: {e}")
#         return None

#     return style_model

# def apply_pretrained_style(content_img, style_name):
#     """Apply a pretrained style to the content image."""

#     print(f"\nApplying pretrained '{style_name}' style...")

#     # Load the pretrained model
#     model = load_pretrained_style_model(style_name)

#     if model is None:
#         print("Failed to load pretrained model")
#         return None

#     # Set to evaluation mode
#     model.eval()

#     # Process the image
#     with torch.no_grad():
#         output = model(content_img)

#     # Convert to image
#     result_img = tensor_to_image(output)

#     return result_img


# def main():
#     """Run style transfer with multiple methods."""

#     # Since content images failed to download, let's create some manually
#     print("Downloading additional images for content...")
#     content_urls = [
#         ("landscape.jpg", "https://upload.wikimedia.org/wikipedia/commons/d/dd/Lake-and-landscape.jpg_-_panoramio.jpg"),
#         ("portrait.jpg", "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/800px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"),
#         ("beach.jpg", "https://www.google.com/imgres?q=sea%20images&imgurl=https%3A%2F%2Fstatic.vecteezy.com%2Fsystem%2Fresources%2Fpreviews%2F015%2F275%2F318%2Fnon_2x%2Fbeautiful-beach-tropical-beach-background-as-summer-landscape-white-sand-and-calm-sea-for-beach-banner-perfect-beach-scene-vacation-and-summer-holiday-concept-boost-up-color-process-photo.jpg&imgrefurl=https%3A%2F%2Fwww.vecteezy.com%2Ffree-photos%2Fsea&docid=otMb8OOdn4XEOM&tbnid=-F5bfAOkKSaIoM&vet=12ahUKEwjy7fmbm_SLAxVmGxAIHTZfARIQM3oFCIMBEAA..i&w=1742&h=980&hcb=2&ved=2ahUKEwjy7fmbm_SLAxVmGxAIHTZfARIQM3oFCIMBEAA")
#     ]

#     for filename, url in content_urls:
#         try:
#             print(f"Downloading {filename} for content...")
#             response = requests.get(url)
#             img_path = os.path.join("data/content", filename)
#             with open(img_path, "wb") as f:
#                 f.write(response.content)
#             print(f"Saved to {img_path}")
#         except Exception as e:
#             print(f"Error downloading {filename}: {e}")

#     # Refresh content image list
#     global content_images
#     content_images = [f for f in os.listdir("data/content") if f.endswith(('.jpg', '.jpeg', '.png'))]
#     print("\nContent images:")
#     print(content_images)

#     # Check if content images are available
#     if not content_images:
#         print("Warning: No content images downloaded. Using style images as content.")
#         # Use some style images as content
#         content_paths = [os.path.join("data/style", img) for img in style_images[:2]]
#         style_paths = [os.path.join("data/style", img) for img in style_images[2:3]]
#     else:
#         # Select random content and style images
#         content_path = os.path.join("data/content", content_images[0])
#         style_path = os.path.join("data/style", style_images[0])
#         content_paths = [content_path]
#         style_paths = [style_path]

#     content_path = content_paths[0]
#     style_path = style_paths[0]

#     print(f"\nContent image: {content_path}")
#     print(f"Style image: {style_path}")

#     # Load images
#     content_img = load_image(content_path)
#     style_img = load_image(style_path)

#     # Store inputs and results
#     results = []

#     # Save original images
#     content_pil = tensor_to_image(content_img)
#     style_pil = tensor_to_image(style_img)
#     content_pil.save("results/content.jpg")
#     style_pil.save("results/style.jpg")

#     results.append(("Content", "results/content.jpg"))
#     results.append(("Style", "results/style.jpg"))

#     # 1. Classic Neural Style Transfer (Gatys et al.)
#     # Reduce the number of steps for faster testing
#     result_gatys = classic_style_transfer(content_img, style_img, num_steps=150)
#     result_gatys.save("results/result_gatys.jpg")
#     results.append(("Gatys et al.", "results/result_gatys.jpg"))

#     # 2. Try to use a pretrained Fast Neural Style Transfer model
#     pretrained_styles = [ "mosaic", "candy", "rain_princess", "udnie", "wave", "tokyo_ghoul"]
#     selected_style = random.choice(pretrained_styles)

#     result_pretrained = apply_pretrained_style(content_img, selected_style)
#     if result_pretrained is not None:
#         result_pretrained.save(f"results/result_pretrained_{selected_style}.jpg")
#         results.append((f"Pretrained ({selected_style})", f"results/result_pretrained_{selected_style}.jpg"))

#     # 3. AdaIN Style Transfer
#     result_adain = adain_style_transfer(content_img, style_img)
#     result_adain.save("results/result_adain.jpg")
#     results.append(("AdaIN", "results/result_adain.jpg"))

#     # 4. Train a Fast Neural Style Transfer model (Johnson et al.)
#     # Note: This is time-consuming, so we'll only do it if explicitly requested
#     train_fast = False
#     if train_fast:
#         transformer = train_fast_style_transfer(style_img, "data/content", epochs=2)
#         result_fast = apply_fast_style_transfer(content_img, transformer)
#         result_fast.save("results/result_fast.jpg")
#         results.append(("Fast NST (Johnson et al.)", "results/result_fast.jpg"))

#     # Display results
#     print("\nResults saved to 'results' directory")

#     # Create a comparison grid
#     fig, axes = plt.subplots(2, len(results)//2 + len(results)%2, figsize=(15, 8))
#     axes = axes.flatten()

#     for i, (title, img_path) in enumerate(results):
#         img = Image.open(img_path)
#         axes[i].imshow(np.array(img))
#         axes[i].set_title(title)
#         axes[i].axis('off')

#     # Remove unused subplots
#     for i in range(len(results), len(axes)):
#         fig.delaxes(axes[i])

#     plt.tight_layout()
#     plt.savefig("results/comparison.jpg", dpi=300)
#     plt.show()

#     print("Style transfer comparison complete!")

# if __name__ == "__main__":
#     main()

Using device: cuda
Error downloading/extracting: File is not a zip file
Saved to data/style/starry_night.jpg
Saved to data/style/the_scream.jpg
Saved to data/style/picasso.jpg
Saved to data/style/mona_lisa.jpg
Saved to data/style/kandinsky.jpg
Saved to data/style/great_wave.jpg
Saved to data/style/candy.jpg

Content images:
['portrait.jpg', 'beach.jpg', 'landscape.jpg']

Style images:
['mona_lisa.jpg', 'picasso.jpg', 'great_wave.jpg', 'starry_night.jpg', 'the_scream.jpg', 'candy.jpg', 'kandinsky.jpg']
Saved to data/content/landscape.jpg
Saved to data/content/portrait.jpg
Saved to data/content/beach.jpg

Content images:
['portrait.jpg', 'beach.jpg', 'landscape.jpg']

Content image: data/content/portrait.jpg
Style image: data/style/mona_lisa.jpg


NameError: name 'load_image' is not defined

In [4]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import requests
from io import BytesIO

# Set random seed for reproducibility
tf.random.set_seed(42)

# Function to load and preprocess images
def load_img(path_to_img, max_dim=512):
    """Load and preprocess images to make them suitable for the model"""
    # Handle different input types
    if isinstance(path_to_img, str):
        if path_to_img.startswith('http'):
            # Download image from URL using requests with a proper User-Agent
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            try:
                response = requests.get(path_to_img, headers=headers, timeout=10)
                response.raise_for_status()  # Check if the request was successful

                # Use PIL to open the image
                img = PIL.Image.open(BytesIO(response.content))
                # Convert to numpy array and then to tensor
                img = np.array(img)
                img = tf.convert_to_tensor(img)
            except Exception as e:
                print(f"Error downloading or processing image from {path_to_img}: {str(e)}")
                raise
        else:
            try:
                img = tf.io.read_file(path_to_img)
                img = tf.image.decode_image(img, channels=3)
            except Exception as e:
                print(f"Error loading local image {path_to_img}: {str(e)}")
                raise
    else:
        img = path_to_img  # Assume it's already a tensor

    # Ensure the image has 3 channels (RGB)
    if img.shape[-1] == 4:  # If RGBA (with alpha channel)
        img = img[..., :3]  # Keep only RGB

    img = tf.image.convert_image_dtype(img, tf.float32)

    # Resize the image while preserving aspect ratio
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

# Function to show images
def imshow(image, title=None):
    """Display an image with an optional title"""
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')

# Function to save images
def save_img(image, filename):
    """Save the generated image to disk"""
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)
    image = tf.clip_by_value(image, 0.0, 1.0)
    image = tf.image.convert_image_dtype(image, tf.uint8)
    image = PIL.Image.fromarray(np.array(image))
    image.save(filename)
    print(f"Image saved to {filename}")

# Function to use TensorFlow Hub model
def style_transfer_hub(content_path, style_path, output_path):
    """Use a pre-trained TensorFlow Hub model for style transfer"""
    try:
        # Load the model
        hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

        # Load images
        content_image = load_img(content_path, max_dim=512)
        style_image = load_img(style_path, max_dim=256)

        # Generate styled image
        outputs = hub_model(tf.constant(content_image), tf.constant(style_image))
        stylized_image = outputs[0]

        # Save the image
        save_img(stylized_image, output_path)
        return stylized_image
    except Exception as e:
        print(f"Error during style transfer: {str(e)}")
        raise

# Main execution
def main():
    # Create output directory
    os.makedirs('output', exist_ok=True)

    # Define content and style image URLs - using more reliable image URLs
    content_images = [
        "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/800px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"
    ]

    style_images = [
        "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/The_Great_Wave_off_Kanagawa.jpg/1024px-The_Great_Wave_off_Kanagawa.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/800px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"
    ]

    print(f"Using {len(content_images)} content images and {len(style_images)} style images from online sources")

    # Run style transfer using TensorFlow Hub model for all combinations
    print("\nRunning TensorFlow Hub style transfer for selected combinations...")

    # Process a few combinations
    combinations = [
        (0, 0),  # Van Gogh's Starry Night with The Great Wave style
        (1, 0)   # Mona Lisa with The Great Wave style
    ]

    for i, j in combinations:
        output_path = f"output/stylized_content{i}_style{j}.jpg"
        print(f"\nApplying style {j} to content {i}...")
        try:
            stylized_hub = style_transfer_hub(content_images[i], style_images[j], output_path)
            print(f"Successfully created stylized image at {output_path}")
        except Exception as e:
            print(f"Failed to process combination {i},{j}: {str(e)}")

    print("\nStyle transfer complete! Results saved in the 'output' directory.")

if __name__ == "__main__":
    main()

Using 2 content images and 2 style images from online sources

Running TensorFlow Hub style transfer for selected combinations...

Applying style 0 to content 0...
Image saved to output/stylized_content0_style0.jpg
Successfully created stylized image at output/stylized_content0_style0.jpg

Applying style 0 to content 1...
Image saved to output/stylized_content1_style0.jpg
Successfully created stylized image at output/stylized_content1_style0.jpg

Style transfer complete! Results saved in the 'output' directory.


## **Task 5-3**

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# ---------------------------
# 1. Hyperparameters
# ---------------------------
BATCH_SIZE = 64
LR = 0.001
EPOCHS = 5

# ---------------------------
# 2. Data Preparation
# ---------------------------
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

# Download CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# ---------------------------
# 3. Define a Simple CNN Model
# ---------------------------
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

        # Activation & dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        # 1st conv block
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)  # 32 -> 16 (image size)

        # 2nd conv block
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)  # 16 -> 8 (image size)

        # Flatten
        x = x.view(x.size(0), -1)  # 64 * 8 * 8 = 4096
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        return x

# Instantiate the model, define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ---------------------------
# 4. Training
# ---------------------------
train_losses = []
train_accuracies = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Calculate training accuracy
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = 100.0 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

# ---------------------------
# 5. Evaluation on Test Set
# ---------------------------
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_accuracy = 100.0 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# ---------------------------
# 6. Plot Training Loss & Accuracy
#    (Each in its own chart)
# ---------------------------

# Plot training loss
# plt.figure()
# plt.plot(range(1, EPOCHS+1), train_losses, marker='o')
# plt.title("Training Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.show()

# # Plot training accuracy
# plt.figure()
# plt.plot(range(1, EPOCHS+1), train_accuracies, marker='o')
# plt.title("Training Accuracy")
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy (%)")
# plt.show()

plt.figure()
plt.plot(range(1, EPOCHS+1), train_losses, marker='o')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.savefig("training_loss.png")  # saves the figure
plt.show()

plt.figure()
plt.plot(range(1, EPOCHS+1), train_accuracies, marker='o')
plt.title("Training Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.savefig("training_accuracy.png")  # saves the figure
plt.show()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Loss: 1.5587, Accuracy: 42.92%
Epoch [2/5], Loss: 1.2411, Accuracy: 55.40%
Epoch [3/5], Loss: 1.1147, Accuracy: 60.37%
Epoch [4/5], Loss: 1.0376, Accuracy: 63.22%
Epoch [5/5], Loss: 0.9905, Accuracy: 65.20%
Test Accuracy: 67.97%


In [6]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

# Set up model paths and configurations
CONFIDENCE_THRESHOLD = 0.5

class ParkingSpaceDetector:
    def __init__(self):
        # Load YOLOv5 model
        print("Loading YOLOv5 model...")
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, trust_repo=True)

        # Configure model settings
        self.model.conf = CONFIDENCE_THRESHOLD
        self.model.classes = [2, 5, 7]  # Filter for cars, buses, trucks

        # Load parking space configurations
        self.parking_spaces = self.load_parking_spaces()

        # Initialize results storage
        self.results_history = []

    def load_parking_spaces(self):
        """
        Load parking space coordinates from config file.
        For demonstration, we'll use fixed coordinates.
        In production, these would be loaded from a JSON file.
        """
        # Example parking space coordinates (x1, y1, x2, y2)
        # In production, these would be defined during setup phase
        return [
            {"id": 1, "coords": (100, 200, 150, 250)},
            {"id": 2, "coords": (160, 200, 210, 250)},
            {"id": 3, "coords": (220, 200, 270, 250)},
            {"id": 4, "coords": (280, 200, 330, 250)},
            {"id": 5, "coords": (100, 260, 150, 310)},
            {"id": 6, "coords": (160, 260, 210, 310)},
            {"id": 7, "coords": (220, 260, 270, 310)},
            {"id": 8, "coords": (280, 260, 330, 310)},
        ]

    def process_frame(self, frame):
        """Process a single video frame to detect parking space occupancy"""
        # Run YOLOv5 on the frame
        results = self.model(frame)

        # Get the detected vehicles
        vehicles = results.pandas().xyxy[0]

        # Check each parking space
        occupancy_status = {}
        for space in self.parking_spaces:
            space_id = space["id"]
            x1, y1, x2, y2 = space["coords"]

            # Check if any vehicle overlaps with this parking space
            is_occupied = False

            for _, vehicle in vehicles.iterrows():
                v_x1, v_y1, v_x2, v_y2 = vehicle[['xmin', 'ymin', 'xmax', 'ymax']]

                # Calculate overlap (IoU - Intersection over Union)
                overlap = self.calculate_iou(
                    (x1, y1, x2, y2),
                    (v_x1, v_y1, v_x2, v_y2)
                )

                if overlap > 0.2:  # If overlap is significant
                    is_occupied = True
                    break

            occupancy_status[space_id] = is_occupied

        # Store result with timestamp
        self.results_history.append({
            "timestamp": datetime.now().isoformat(),
            "occupancy": occupancy_status
        })

        return occupancy_status, frame

    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union between two bounding boxes"""
        # Determine intersection coordinates
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        # Calculate area of intersection
        width = max(0, x2 - x1)
        height = max(0, y2 - y1)
        intersection_area = width * height

        # Calculate areas of both boxes
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

        # Calculate IoU
        union_area = box1_area + box2_area - intersection_area

        if union_area == 0:
            return 0

        return intersection_area / union_area

    def visualize_results(self, frame, occupancy_status):
        """Draw parking spaces and their status on the frame"""
        result_frame = frame.copy()

        for space in self.parking_spaces:
            space_id = space["id"]
            x1, y1, x2, y2 = space["coords"]

            # Choose color based on occupancy (green for empty, red for occupied)
            color = (0, 255, 0) if not occupancy_status[space_id] else (0, 0, 255)

            # Draw rectangle for parking space
            cv2.rectangle(result_frame, (x1, y1), (x2, y2), color, 2)

            # Add space ID label
            cv2.putText(result_frame, str(space_id), (x1 + 5, y1 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Add summary information
        available_count = sum(1 for status in occupancy_status.values() if not status)
        total_count = len(occupancy_status)

        cv2.putText(result_frame, f"Available: {available_count}/{total_count}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return result_frame

    def get_occupancy_stats(self):
        """Generate statistics about parking space usage over time"""
        if not self.results_history:
            return "No data available yet"

        # Calculate how often each space is occupied
        space_ids = [space["id"] for space in self.parking_spaces]
        occupancy_rates = {space_id: 0 for space_id in space_ids}

        for record in self.results_history:
            occupancy = record["occupancy"]
            for space_id, is_occupied in occupancy.items():
                if is_occupied:
                    occupancy_rates[space_id] += 1

        # Convert to percentages
        total_records = len(self.results_history)
        for space_id in occupancy_rates:
            occupancy_rates[space_id] = (occupancy_rates[space_id] / total_records) * 100

        return occupancy_rates

def download_sample_images():
    """
    Download sample parking lot images from the internet for demonstration
    Returns a list of images as numpy arrays
    """
    print("Downloading sample parking lot images...")

    # URLs for sample parking lot images
    image_urls = [
        "https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-224.jpg",
        "https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-303.jpg",
        "https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-407.jpg",
        "https://cdn.shopify.com/s/files/1/0196/4614/articles/How_can_Smart_Parking_Solutions_help_Solve_Parking_Problems_1400x.png",
        "https://images.unsplash.com/photo-1525126997622-d5c1bb293dfe"
    ]

    images = []

    for url in image_urls:
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Convert to image and then to numpy array
                img = Image.open(BytesIO(response.content))
                img_array = np.array(img)

                # Convert to RGB if needed
                if len(img_array.shape) == 2:  # Grayscale image
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                elif img_array.shape[2] == 4:  # RGBA image
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_RGBA2RGB)

                images.append(img_array)
                print(f"Successfully downloaded image from {url}")
            else:
                print(f"Failed to download image from {url}: Status code {response.status_code}")
        except Exception as e:
            print(f"Error downloading image from {url}: {e}")

    if not images:
        # Generate synthetic images if no real images could be downloaded
        print("Generating synthetic parking lot images...")
        for _ in range(3):
            # Create a simple synthetic image
            img = np.ones((500, 800, 3), dtype=np.uint8) * 120  # Gray background

            # Draw some parking spaces
            for i in range(4):
                for j in range(6):
                    x = 100 + j * 100
                    y = 100 + i * 80
                    # Randomly decide if parking space is occupied
                    if np.random.rand() > 0.5:
                        # Draw a car (rectangle)
                        cv2.rectangle(img, (x, y), (x + 80, y + 60), (0, 0, 200), -1)
                    else:
                        # Draw empty parking space
                        cv2.rectangle(img, (x, y), (x + 80, y + 60), (100, 100, 100), 2)

            images.append(img)

    print(f"Total images ready for processing: {len(images)}")
    return images

def demo_with_sample_images(detector):
    """Run detection on a set of sample images from the web or synthetic images"""
    # Get sample images
    sample_images = download_sample_images()

    plt.figure(figsize=(15, 10))

    for i, img in enumerate(sample_images[:3]):  # Process at most 3 images for demo
        # Process frame
        occupancy, _ = detector.process_frame(img)

        # Visualize results
        result_frame = detector.visualize_results(img, occupancy)

        # Display
        plt.subplot(len(sample_images[:3]), 1, i+1)
        plt.imshow(result_frame)
        plt.title(f"Sample {i+1}")

    plt.tight_layout()
    plt.savefig('parking_detection_results.png')
    plt.show()

    # Print occupancy stats
    print("\nParking Space Occupancy Rates:")
    occupancy_rates = detector.get_occupancy_stats()

    if isinstance(occupancy_rates, dict):
        for space_id, rate in occupancy_rates.items():
            print(f"Space {space_id}: {rate:.1f}% occupied")
    else:
        print(occupancy_rates)  # In case it returns a message like "No data available yet"

# Usage example for demo
if __name__ == "__main__":
    print("Initializing Parking Space Detector...")
    # Initialize detector
    detector = ParkingSpaceDetector()

    # Run the demo
    print("Running detection on sample images...")
    demo_with_sample_images(detector)

    print("Demo complete!")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-6 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)



Initializing Parking Space Detector...
Loading YOLOv5 model...


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Running detection on sample images...
Failed to download image from https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-224.jpg: Status code 404
Failed to download image from https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-303.jpg: Status code 404
Failed to download image from https://raw.githubusercontent.com/mbeyeler/opencv-python-blueprints/master/chapter4/images/parking-lot-img-407.jpg: Status code 404
Failed to download image from https://cdn.shopify.com/s/files/1/0196/4614/articles/How_can_Smart_Parking_Solutions_help_Solve_Parking_Problems_1400x.png: Status code 404
Failed to download image from https://images.unsplash.com/photo-1525126997622-d5c1bb293dfe: Status code 404
Generating synthetic parking lot images...
Total images ready for processing: 3

Parking Space Occupancy Rates:
Space 1: 0.0% occupied
Space 2: 0.0% occupied
Space 3: 0.0% occupied
Space 4: 0.0% occ